In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
import numpy as np
import pandas as pd
import text_normalizer as tn
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import warnings
import model_report as mr
warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
file_info = "C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\cleaned_chunked_v2.csv"
df = pd.read_csv(file_info, sep='\t')

In [6]:
df.head()

,url,code,category,language,title,summary,key_words,content,combined,nouns,compounds,comp_nouns,flat_comp_nouns,clean_comp_nouns
0,*://BLAZINHERB.CA,200,dr,en,Blazin Herb,The Diamond High Crew,Blazin herb,Blazin Herb THC Edibles Concentrates Flower/bu...,Blazin Herb The Diamond High Crew Blazin herb ...,"[('Blazin', 0, 6, 'PROPN'), ('Herb', 7, 11, 'P...","[('Blazin Herb', 0, 11, 'COMPOUND'), ('Diamond...","{'Read', 'Picks', 'DIAMOND', 'bud', 'THC', 'So...","{'Read', 'Picks', 'DIAMOND', 'bud', 'THC', 'So...",read picks diamond bud thc social sleep club r...
1,*://THEHONEYPOTSTORE.CA,200,dr,en,The Honey Pot Cannabis and Medical Marijuana p...,"The Honey Pot Hamilton Ontario, provides canna...","The Honey Pot ,Georgia Peach Cannabis and Medi...",The Honey Pot Home About Us F.A.Q.'s About The...,The Honey Pot Cannabis and Medical Marijuana p...,"[('Honey', 4, 9, 'PROPN'), ('Pot', 10, 13, 'PR...","[('Honey Pot Cannabis', 4, 22, 'COMPOUND'), ('...","{'appetite', 'Stress', 'Muscle', 'Hepatitis C'...","{'appetite', 'Stress', 'Muscle', 'Hepatitis C'...",appetite stress muscle hepatitis c hamilton hi...
2,*://AMANTE.CA,200,dr,en,Amante Organics - Licensed Certified Organic C...,np-template-header-footer-from-plugin,"Organic Cannabis, Certified Organic, Weed, Org...",Amante Organics LOCAL | ORGANIC | SUSTAINABLE ...,Amante Organics - Licensed Certified Organic C...,"[('Amante', 0, 6, 'PROPN'), ('Organics', 7, 15...","[('Amante Organics -', 0, 17, 'COMPOUND'), ('O...","{'nations', 'Rights Reserved', ""a'mante"", 'Lic...","{'nations', 'Rights Reserved', ""a'mante"", 'Lic...",nation rights reserved amante licensed np head...
3,*://CANNAFLY.CA,200,dr,en,Cannafly,Cannafly,"Marijuana, cannabis, medical marijuana, dispen...",Vancouver's Premium Mobile Medicinal Cannabis ...,"Cannafly Cannafly Marijuana, cannabis, medical...","[('Cannafly', 0, 8, 'PROPN'), ('Cannafly', 9, ...","[('marijuana legalization', 70, 92, 'COMPOUND'...","{'chilliwack', 'Connoisseurs', 'THC', 'industr...","{'chilliwack', 'Connoisseurs', 'THC', 'industr...",chilliwack connoisseurs thc industry death bub...
4,*://GRON.VC,200,dr,en,Grøn Ventures,We are backing businesses and brands with pote...,"cannabis vc, cannabis venture capital, gron ve...","GRØN VENTURES 4675 MacArthur Ct, 15th Floor, N...",Grøn Ventures We are backing businesses and br...,"[('Grøn', 0, 4, 'PROPN'), ('Ventures', 5, 13, ...","[('Grøn Ventures', 0, 13, 'COMPOUND'), ('marke...","{'marijuana', 'Beach', 'Newport', 'ventures', ...","{'marijuana', 'Beach', 'Newport', 'ventures', ...",marijuana beach newport venture ct ventures ma...


In [16]:
import nltk
stopword_list = nltk.corpus.stopwords.words('english')
#keep negation if in any bi-grams
stopword_list.remove('no')
stopword_list.remove('not')

#normalize the corpus
norm_corpus = tn.normalize_corpus(corpus=df['combined'],
                                  html_stripping=False,
                                  contraction_expansion=True,
                                  text_lemmatization=True,
                                  text_stemming=False,
                                  special_char_removal=True,
                                  remove_digits=True,
                                  stopword_removal=True,
                                  stopwords=stopword_list
                                 )
df['clean_combined'] = norm_corpus

In [43]:
import yake

language = "en"
max_ngram_size = 2
deduplication_threshold = 0.2
numOfkeywords = 10
kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfkeywords,features=None)

#find the key words
df['key_words'] = ""
for row in range(len(df)):
    keywords =  kw_extractor.extract_keywords(df.loc[row, 'clean_combined'])
    a, b = zip(*keywords)
    df.loc[row,'key_words'] = str(a)



In [44]:
df[['clean_combined','key_words']]

,clean_combined,key_words
0,blazin herb diamond high crew blazin herb blaz...,"('blazin herb', 'high club', 'edibles concentr..."
1,honey pot cannabis medical marijuana product s...,"('honey pot', 'gift certificate', 'medical mar..."
2,amante organics licensed certified organic can...,"('cannabis producer', 'certified organic', 'pr..."
3,cannafly cannafly marijuana cannabis medical m...,"('minimum order', 'view detail', 'hybrid view'..."
4,grn ventures back business brand potential mar...,"('grn ventures', 'back business', 'potential m..."
...,...,...
41871,e r r n taranaki educational resource research...,"('educational resource', 'research analysis', ..."
41872,party pills legal highs partypills party pill ...,"('party pills', 'legal high', 'pills herbal', ..."
41873,introduce yoga pilates salus auckland new zeal...,"('yoga pilates', 'salus yoga', 'pilate aucklan..."
41874,home aim program provide foundation student wa...,"('sokoine university', 'university agriculture..."


In [45]:
file_out = "C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\cleaned_combined_key_words_v1.csv"
df.to_csv(file_out,index=False,sep='\t')

In [46]:
import nltk
stopword_list = nltk.corpus.stopwords.words('english')
#keep negation if in any bi-grams
stopword_list.remove('no')
stopword_list.remove('not')

#normalize the corpus
norm_corpus = tn.normalize_corpus(corpus=df['key_words'],
                                  html_stripping=False,
                                  contraction_expansion=True,
                                  text_lemmatization=True,
                                  text_stemming=False,
                                  special_char_removal=True,
                                  remove_digits=True,
                                  stopword_removal=True,
                                  stopwords=stopword_list
                                 )
df['clean_key_words'] = norm_corpus

In [47]:
df[['clean_combined','clean_key_words']]

,clean_combined,clean_key_words
0,blazin herb diamond high crew blazin herb blaz...,blazin herb high club edible concentrate flowe...
1,honey pot cannabis medical marijuana product s...,honey pot gift certificate medical marijuana h...
2,amante organics licensed certified organic can...,cannabis producer certify organic producer alb...
3,cannafly cannafly marijuana cannabis medical m...,minimum order view detail hybrid view hope min...
4,grn ventures back business brand potential mar...,grn venture back business potential market lea...
...,...,...
41871,e r r n taranaki educational resource research...,educational resource research analysis analysi...
41872,party pills legal highs partypills party pill ...,party pill legal high pill herbal good party h...
41873,introduce yoga pilates salus auckland new zeal...,yoga pilate salus yoga pilate auckland fitness...
41874,home aim program provide foundation student wa...,sokoine university university agriculture aim ...


In [48]:
file_out = "C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\cleaned_combined_key_words_v1.csv"
df.to_csv(file_out,index=False,sep='\t')